In [3]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("output.csv")
docs = loader.load()
docs

[Document(page_content='Course: The Complete Python Bootcamp From Zero to Hero in PythonLearn Python like a Professional  Start from the basics and go all the way to creating your own applications and gamesRating: 4.6 out of 5502086 reviews22 total hours156 lecturesAll LevelsCurrent price: â‚¹499Original price: â‚¹3,099\nLink: https://www.udemy.com/course/complete-python-bootcamp/\nCourse Name: The Complete Python Bootcamp From Zero to Hero in Python\ntotal hours: 22.0\nrating: 4.6\nlevel: All Levels', metadata={'source': 'output.csv', 'row': 0}),
 Document(page_content='Course: The Complete 2024 Web Development BootcampBecome a Full-Stack Web Developer with just ONE course. HTML, CSS, Javascript, Node, React, PostgreSQL, Web3 and DAppsRating: 4.7 out of 5371464 reviews61.5 total hours373 lecturesAll LevelsCurrent price: â‚¹499Original price: â‚¹3,099\nLink: https://www.udemy.com/course/the-complete-web-development-bootcamp/\nCourse Name: The Complete 2024 Web Development Bootcamp\ntot

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 10)
documents = text_splitter.split_documents(docs)

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [6]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [7]:
db

In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model= "llama2")
llm

Ollama()

In [9]:
#chat prompt

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """ Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
    context and given skills. and also provide Course link.
    <context>
    {context}
    <context>

    skills : {input}
"""
)

In [10]:
#stuff chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
#retriever

retriever = db.as_retriever()

In [12]:
#retriever chain

from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)

In [13]:
response = retriever_chain.invoke({"input": "my skills are React and java"})

In [14]:
response['answer']

"Based on the skills you provided (React and Java), I would recommend the following courses to help you enhance your knowledge and skills:\n\n1. The Complete JavaScript Course 2024: From Zero to Expert! - This course is an excellent starting point for learning JavaScript, as it covers everything from the basics to advanced concepts. With a rating of 4.7 out of 5 and 321 lectures, you'll have plenty of material to work through. Plus, it includes projects and challenges to help you apply your knowledge in real-time.\n2. The Complete Node.js Developer Course (3rd Edition) - If you're interested in learning Node.js, this course is a great choice. It covers everything from the basics of Node.js to advanced topics like Express and MongoDB. With 177 lectures and a rating of 4.5 out of 5, you can be sure that you're getting high-quality content.\n3. Selenium WebDriver with Java - This course is perfect for those interested in learning how to use Selenium with Java. With a rating of 4.6 out of 

### EX-2

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [16]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever, input = RunnablePassthrough())

In [17]:
chain = setup | prompt | llm | parser

In [18]:
chain.invoke("my skills are React and java")

"Great! Based on your skills in React and Java, I would recommend the following courses to you:\n\n1. The Complete JavaScript Course 2024: This course is a comprehensive introduction to JavaScript, covering both the basics and advanced concepts. It's a great starting point for beginners and experienced developers alike, and it includes projects and challenges to help you practice your skills.\n2. Selenium WebDriver with Java - Basics to Advanced + Frameworks: This course is focused specifically on Selenium and Java, which are both relevant to your skills in React and Java. It covers the basics of Selenium as well as more advanced topics and frameworks, making it a great choice for those looking to deepen their understanding of these technologies.\n3. The Complete Node.js Developer Course (3rd Edition): While not specifically focused on React or Java, this course covers the foundational concepts of Node.js development, which can be useful for anyone interested in building web applicatio

## EX-03

In [19]:
from langchain_core.output_parsers import JsonOutputParser

parser_js = JsonOutputParser()

In [28]:
#chat prompt



prompt_json= ChatPromptTemplate.from_template(
    """ Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
    context and given skills. also provide the course link, total hours and rating.
    Take your time think step by step before providing detail answer. 
    If there is a course suggestion, extract as a Json with keys as follows:
        - "Course Name": Name of the course.
        - "Course Link" : Link of the course.
        - "total hours": Total hours of the course.
    
    Strickly follow the context.
    <context>
    {context}
    <context>

    skills : {input}
"""
)

In [21]:
chain_js = setup | prompt_json | llm | parser_js

In [22]:
chain_js.invoke("my skills are React and java")

OutputParserException: Invalid json output: Based on your provided context and skills, I would suggest the following courses as the best fit for you:

1. The Complete JavaScript Course 2024 - This course covers a wide range of topics in JavaScript, including projects, challenges, and theory. As someone with skills in React and Java, this course will provide you with a comprehensive understanding of JavaScript and its applications. With a rating of 4.7 out of 5 and 321 lectures, this course is an excellent choice for learning the latest techniques and best practices in JavaScript. (Total hours: 68.5)
2. Selenium WebDriver with Java - This course covers the basics to advanced levels of Selenium WebDriver with Java. As a beginner or intermediate learner, this course will provide you with hands-on experience in using Selenium for web automation. With a rating of 4.6 out of 5 and 335 lectures, this course is an excellent choice for learning the latest techniques and best practices in Seleni

In [23]:
chain_js = setup | prompt_json | llm

In [24]:
chain_js.invoke("my skills are React and java")

'Great! Based on the information provided, I would recommend the following courses to help you improve your skills in React and Java:\n\n1. The Complete JavaScript Course 2024: From Zero to Expert! - This course covers a wide range of topics in JavaScript, including React and Java. With over 321 lectures and 68.5 total hours, this course is an excellent resource for learning the basics of JavaScript as well as advanced concepts. The rating for this course is 4.7 out of 5, with over 200,000 students enrolled.\n2. Selenium WebDriver with Java - Basics to Advanced + Frameworks - This course focuses specifically on Selenium and Java, which are important skills for a React developer. With over 335 lectures and 55.5 total hours, this course covers the basics of Selenium as well as advanced concepts and frameworks. The rating for this course is 4.6 out of 5, with over 1 million students enrolled.\n3. The Complete Node.js Developer Course (3rd Edition) - While not specifically focused on React

In [31]:
chain_js_ds = setup | prompt_json | llm | parser_js

In [32]:
chain_js_ds.invoke("I am a Data Scientiest with  over 2 years of experience I know Machine Learning, Deep Learning and Python")

{'Course Name': 'The Complete JavaScript Course 2024',
 'Course Link': '<https://www.udemy.com/course/the-complete-javascript-course/>',
 'Total Hours': 68.5,
 'Rating': 4.7}

In [33]:
chain_js_ds.invoke("my skills are Machine Learning, Deep Learning and Python")

OutputParserException: Invalid json output: Based on the provided context and your listed skills, I would recommend the following courses:

1. The Complete Java Course 2024 - This course is a comprehensive introduction to Java programming, covering both theoretical and practical aspects of the language. As a Machine Learning and Deep Learning enthusiast, you may find this course helpful in understanding the basics of Java programming, which can be useful for implementing machine learning algorithms. The course has a rating of 4.7 out of 5 and includes 321 lectures, spanning 68.5 hours.
2. Selenium WebDriver with Java - This course is designed to teach you the basics of Selenium WebDriver programming using Java. As a Python enthusiast, you may find this course helpful in learning how to automate web applications using Selenium. The course has a rating of 4.6 out of 5 and includes 335 lectures, spanning 55.5 hours.
3. The Complete Python Bootcamp From Zero to Hero in Python - This course